# Homework 2: ONNX Accuracy & Size

## Import the required modules

In [1]:
import numpy as np
import onnxruntime as ort
import os
import zipfile

from msc_dataset import MSCDataset

## Create the Test Dataset

In [2]:
CLASSES = ['stop', 'up']

test_ds = MSCDataset(
    root='.',
    classes=CLASSES,
    split='testing',
    preprocess=None
)
print(f"Test dataset size: {len(test_ds)}")

Using data folder: ./msc-testing
Loaded 200 samples from ./msc-testing for classes ['stop', 'up']
Test dataset size: 200


## Set ONNX files name

In [3]:
# Trova automaticamente l'ultimo modello salvato
import glob

saved_models_dir = './saved_models/'
frontend_files = sorted(glob.glob(f'{saved_models_dir}*_frontend.onnx'))
model_files = sorted(glob.glob(f'{saved_models_dir}*_model.onnx'))

if not frontend_files or not model_files:
    raise FileNotFoundError("No ONNX models found in ./saved_models/")

# Prendi l'ultimo (ordinato per timestamp nel nome)
frontend_file = frontend_files[-1]
model_file = model_files[-1]

print(f"Using models:")
print(f"  Frontend: {frontend_file}")
print(f"  Model: {model_file}")

Using models:
  Frontend: ./saved_models/1764339634_frontend.onnx
  Model: ./saved_models/1764339634_model.onnx


## Unzip the ONNX model, if .zip is provided

In [4]:
if model_file.lower().endswith(".zip"):
    with zipfile.ZipFile(model_file, "r") as z:
        # The only file inside the ZIP
        onnx_model_file = z.namelist()[0]
        z.extract(onnx_model_file, ".")   # extract into current directory
else:
    onnx_model_file = model_file

## Evaluate the ONNX Model

In [ ]:

ort_frontend = ort.InferenceSession(frontend_file)
ort_model = ort.InferenceSession(onnx_model_file)

true_count = 0.0
for sample in test_ds:
    inputs = sample['x']
    label = sample['y']  # Changed from 'label' to 'y'
    inputs = inputs.squeeze(0).numpy()  # Remove channel dimension first
    inputs = np.expand_dims(inputs, 0)  # Add batch dimension
    features = ort_frontend.run(None, {'input': inputs})[0]
    outputs = ort_model.run(None,  {'input': features})[0]
    prediction = np.argmax(outputs, axis=-1).item()
    true_count += prediction == label

accuracy = true_count / len(test_ds) * 100
frontend_size = os.path.getsize(frontend_file)
model_size = os.path.getsize(model_file)
total_size = frontend_size + model_size

print(f'🎯 ONNX Accuracy: {accuracy:.2f}%')
print(f'📦 Feature Extraction Size: {frontend_size / 2**10:.1f}KB')
print(f'📦 Model Size: {model_size / 2**10:.1f}KB')
print(f'📦 Total Size: {total_size / 2**10:.1f}KB')

🎯 Accuracy: 90.00%
📦 Feature Extraction Size: 332.5KB
📦 Model Size: 1022.7KB
📦 Total Size: 1355.2KB
